In [ ]:
# default_exp schema

# schema
> checking dictionaries based on json schema

In [ ]:
#export
import jsonschema, requests, yaml
from types import SimpleNamespace

# Get schema from path

In [ ]:
import dpath.util
def getSchemaPath(schemaUrl:str, path:str='/', isYaml = True):
  '''
  get a nested schema from path
  schemaUrl: str: url of the schema
  path: str: path of the schema, if root then path='/'
  isYaml: Bool: is the schema yaml (false indicates that the schema is json)
  '''
  if isYaml:
    schema = yaml.load(requests.get(schemaUrl).text, Loader= yaml.Loader)
  else:
    schema = requests.get(schemaUrl).json()
  return dpath.util.get(schema, path)

In [ ]:
testSchema = 'https://gist.githubusercontent.com/thanakijwanavit/e2720d091ae0cef710a49b57c0c9cd4c/raw/ed2d322eac4900ee0f95b431d0f9067a40f3e0f0/squirrelOpenApiV0.0.3.yaml'
path = 'components/schemas/Location'
getSchemaPath(testSchema, path)

{'type': 'object',
 'required': ['id',
  'type',
  'street_address',
  'city',
  'state',
  'zip',
  'capacity',
  'status'],
 'properties': {'id': {'type': 'string', 'format': 'uuid'},
  'type': {'type': 'string', 'enum': ['pick up', 'drop off', 'overnight']},
  'street_address': {'type': 'string'},
  'city': {'type': 'string'},
  'state': {'type': 'string',
   'pattern': '^(?:(A[KLRZ]|C[AOT]|D[CE]|FL|GA|HI|I[ADLN]|K[SY]|LA|M[ADEINOST]|N[CDEHJMVY]|O[HKR]|PA|RI|S[CD]|T[NX]|UT|V[AT]|W[AIVY]))$'},
  'zip': {'type': 'string', 'pattern': '(^\\d{5}$)|(^\\d{5}-\\d{4}$)'},
  'status': {'type': 'string', 'enum': ['open', 'in use']},
  'created': {'type': 'string', 'format': 'date-time'},
  'modified': {'type': 'string', 'format': 'date-time'}}}

# Validate Url

In [ ]:
#export
def validateUrl(url,input_, format_ = 'json', headers = {'Cache-Control': 'no-cache'}, path = '/'):
  if format_ == 'yaml':
    schema = getSchemaPath(url, path = path, isYaml = True)
  elif format_ == 'json':
    schema = getSchemaPath(url, path = path, isYaml = False)
  else:
    print('invalid schema format, using json')
    schema = requests.get(url).json()
  res = jsonschema.validate(input_,schema)
  return SimpleNamespace(**input_)

### test json

In [ ]:
url = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json'
input_ = {'iprcode': 4, 'cprcode': 123 , 'oprCode': '123'}
validateUrl(url, input_)

namespace(iprcode=4, cprcode=123, oprCode='123')

#### error json

In [ ]:
errorProduct = [{ 'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] #ProductDatabase.valueUpdate({'items':sampleProducts})
url = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/valueUpdate.json'
try:
  validateUrl(url, errorProduct)
except Exception as e:
  print(f'{e}')

[{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] is not of type 'object'

Failed validating 'type' in schema:
    {'$schema': 'http://json-schema.org/draft-06/schema#',
     'definitions': {'Product': {'description': 'Product(iprcode: int, '
                                                'cprcode: int)',
                                 'properties': {'cprcode': {'type': 'integer'},
                                                'iprcode': {'type': 'integer'}},
                                 'required': ['iprcode', 'cprcode'],
                                 'type': 'object'}

### test yaml

In [ ]:
##success
url = 'https://gist.githubusercontent.com/thanakijwanavit/241c4cc443f39ea096820f5dfb84017d/raw/61694a0c5fbac3f6408fbb11217cc4265d38e38d/sampleYaml.yaml'
input_ = {'iprcode': 4, 'cprcode': 123 , 'oprCode': '123'}
print(validateUrl(url, input_, format_='yaml'))
## failure
try:
  print(validateUrl(url, errorProduct, format_='yaml'))
except Exception as e:
  print(f'{e}')

namespace(cprcode=123, iprcode=4, oprCode='123')
[{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUNG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}] is not of type 'object'

Failed validating 'type' in schema:
    {'$schema': 'http://json-schema.org/schema#',
     'properties': {'cprcode': {'type': 'integer'},
                    'iprcode': {'type': 'integer'},
                    'oprCode': {'type': 'string'}},
     'required': ['cprcode', 'iprcode', 'oprCode'],
     'type': 'object'}

On instance:
    [{'cprcode': '0171670',
      'iprcode': '0171670',
      'oprcode': '0171670',
      'ordertype': 'Y',
      'pr_abb': 'JIRAPAT 

## convert type to comply with json schema

In [ ]:
#export
typeMap = {'string': str, 'number': float, 'integer': int, 'object': dict, 'array': list, 'boolean': bool, 'null': None}
def getTypes(schemaUrl:str, typeMap:dict=typeMap)->dict:
  '''get python types from json schema'''
  r = requests.get(schemaUrl)
  s = yaml.load(r.text, Loader=yaml.FullLoader)
  properties = s['properties']
  dtypes = {k: typeMap.get(v['type']) for k,v in properties.items()}
  return dtypes
def typeMapJsonSchema(url:str, input_:dict = {}, typeMap:dict = typeMap, defaultType=str):
  '''
  try to map the datatype into the one specified in url of json schema. 
  if type is not found, the defaultType is used
  '''
  typesDict = getTypes(url, typeMap=typeMap) # get dtype from schema url
  print(f'typesDict is: {typesDict}')
  convertedInput = {k: (typesDict.get(k) or defaultType)(v) for k,v in input_.items()}
  return convertedInput



In [ ]:
url = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/dev-manual/inventory/inventory.yaml'


inv = {
                  'iprcode': '0000009',
                  'brcode': '1000',
                  'ib_cf_qty': '50',
                  'new_ib_vs_stock_cv': '27',
                  'onlineflag': True
                }
typeMapJsonSchema(url, input_=inv)

typesDict is: {'iprcode': <class 'int'>, 'brcode': <class 'int'>, 'ib_cf_qty': <class 'int'>, 'new_ib_vs_stock_cv': <class 'int'>, 'onlineflag': <class 'bool'>}


{'iprcode': 9,
 'brcode': 1000,
 'ib_cf_qty': 50,
 'new_ib_vs_stock_cv': 27,
 'onlineflag': True}

In [ ]:
typeMapJsonSchema(url, input_=inv)

typesDict is: {'iprcode': <class 'int'>, 'brcode': <class 'int'>, 'ib_cf_qty': <class 'int'>, 'new_ib_vs_stock_cv': <class 'int'>, 'onlineflag': <class 'bool'>}


{'iprcode': 9,
 'brcode': 1000,
 'ib_cf_qty': 50,
 'new_ib_vs_stock_cv': 27,
 'onlineflag': True}

# get schema path